In [2]:
from datetime import datetime
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud import bigquery
from google.cloud.exceptions import NotFound
import pandas as pd
import yahoo_fin.stock_info as si
from utils.utils_bigquery import *
from datetime import *

In [3]:
# Ruta credenciales de CGP
key_path = key_path

# Datos de origen de datos en Bigquery
project = project_id
dataset = 'bronze'
dataset_to_save = 'silver'
table = 'bronze_currency_data'
table_to_save = 'silver_currency_data'
table_conca = f'{project}.{dataset}.{table}'

In [4]:
# Conectamos con Bigquery
bigquery = BigQueryUtils(key_path)

In [7]:
# Obtenemos los datos almacenados en Bigquery
df = bigquery.run_query(
    f"""
    SELECT
        *
    FROM {table_conca}
    """
)

In [8]:
# Mostramos primeras filas
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Currency
0,2015-01-01 00:00:00+00:00,0.82654,0.826540,0.826540,0.82654,0.82654,0,USDEUR
1,2015-01-02 00:00:00+00:00,0.82722,0.832584,0.827160,0.82717,0.82717,0,USDEUR
2,2015-01-05 00:00:00+00:00,0.83647,0.841107,0.835010,0.83707,0.83707,0,USDEUR
3,2015-01-06 00:00:00+00:00,0.83764,0.841260,0.835422,0.83759,0.83759,0,USDEUR
4,2015-01-07 00:00:00+00:00,0.84212,0.847170,0.840336,0.84208,0.84208,0,USDEUR


In [9]:
# Convert 'Date' column to datetime type
df['Date'] = pd.to_datetime(df['Date'])

# Pivot the DataFrame
df_pivot = df.pivot_table(
    index='Date',          
    columns='Currency',    
    values='Close',        
    aggfunc='first'        
)

# Reset index to make 'Date' a column again
df_pivot = df_pivot.reset_index()

df_pivot

Currency,Date,USDCHF,USDCNY,USDEUR,USDGBP,USDJPY
0,2015-01-01 00:00:00+00:00,0.99411,6.1961,0.82654,0.64181,119.672997
1,2015-01-02 00:00:00+00:00,0.99410,6.1961,0.82717,0.64186,119.870003
2,2015-01-05 00:00:00+00:00,1.00580,6.1961,0.83707,0.65424,120.433998
3,2015-01-06 00:00:00+00:00,1.00620,6.2084,0.83759,0.65538,119.425003
4,2015-01-07 00:00:00+00:00,1.01130,6.2020,0.84208,0.66059,118.672997
...,...,...,...,...,...,...
2515,2024-08-27 00:00:00+00:00,0.84710,7.1181,0.89568,0.75805,144.516006
2516,2024-08-28 00:00:00+00:00,0.84172,7.1231,0.89462,0.75429,143.975006
2517,2024-08-29 00:00:00+00:00,0.84182,7.1243,0.89888,0.75794,144.432999
2518,2024-08-30 00:00:00+00:00,0.84694,7.0955,0.90249,0.75952,144.889999


In [13]:
# Calculate the number of null values per column
nulls_by_column = df_pivot.isnull().sum()

# Sort the result in descending order
nulls_by_column_sorted = nulls_by_column.sort_values(ascending=False)

print(nulls_by_column_sorted)


Currency
USDCNY    1
Date      0
USDCHF    0
USDEUR    0
USDGBP    0
USDJPY    0
dtype: int64


In [16]:
# Rellenar los valores nulos con el valor anterior
df_pivot['USDCNY'] = df_pivot['USDCNY'].fillna(method='ffill')

# Calculate the number of null values per column
nulls_by_column = df_pivot.isnull().sum()

# Sort the result in descending order
nulls_by_column_sorted = nulls_by_column.sort_values(ascending=False)

nulls_by_column_sorted

Currency
Date      0
USDCHF    0
USDCNY    0
USDEUR    0
USDGBP    0
USDJPY    0
dtype: int64

In [17]:
# Guardamos los datos en bigquery
bigquery.save_dataframe(df_pivot, project, dataset_to_save, table_to_save, if_exists='replace', schema=None)

100%|██████████| 1/1 [00:00<?, ?it/s]
